import modules

In [166]:
%matplotlib inline
from os.path import join as opj
import json
from nipype.interfaces.fsl import FLIRT
from nipype.interfaces.spm import Level1Design, EstimateModel, EstimateContrast
from nipype.interfaces.freesurfer import FSCommand
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.interfaces.utility import Function, IdentityInterface, Merge
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode
import nipype.interfaces.spm as spm
from nipype.interfaces.base import Bunch
from nipype.interfaces.freesurfer import ApplyVolTransform, MRIConvert
from nipype.algorithms.misc import Gunzip

In [167]:
# SPM standalone
matlab_cmd = '/usr/local/spm12/run_spm12.sh /usr/local/MCR/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

specify interfaces behaviors

In [168]:
# MATLAB - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/usr/local/MATLAB/R2014a/toolbox/spm12')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FreeSurfer - Specify the location of the freesurfer folder
fs_dir = '~/nipype_tutorial/freesurfer'
FSCommand.set_default_subjects_dir(fs_dir)

experiment parameters

In [169]:
experiment_dir = '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/'
output_dir = 'output_firstlevel_mask'
working_dir = 'working_firstlevel_mask'

# list of subject identifiers
#subject_list = ['sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-09', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15', 'sub-16', 'sub-17', 'sub-19', 'sub-20', 'sub-21', 'sub-22', 'sub-23', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29' ]
subject_list = ['sub-04']

# TR of functional images
TR = 1.45
number_of_slices = 30
fwhm = [4]

In [170]:
# Condition names
condition_names = ['stress_1', 'relax_1', 'stress_1_gremium','Realign1','Realign2','Realign3','Realign4','Realign5','Realign6','Realign7','Realign8','stress_2', 'relax_2', 'stress_2_gremium']

cont01 = ['stress','T', condition_names, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]*2]
cont02 = ['relax','T', condition_names, [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]*2]
cont03 = ['stress > relax','T', condition_names, [1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0]*2]
cont04 = ['relax > stress','T', condition_names, [-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]*2]
cont05 = ['stress_gremium','T', condition_names, [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]*2]

cont06 = ['stress_1 run 1','T', condition_names, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
cont07 = ['relax_1 run 1','T', condition_names, [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
cont08 = ['stress_1 > relax_1 run 1','T', condition_names, [1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
cont09 = ['relax_1 > stress_1 run 1','T', condition_names, [-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
cont10 = ['stress_1_gremium run 1','T', condition_names, [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

cont11 = ['stress_2 run 2','T', condition_names, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
cont12 = ['relax_2 run 2','T', condition_names, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
cont13 = ['stress_2 > relax_2 run 2','T', condition_names, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
cont14 = ['relax_2 > stress_2 run 2','T', condition_names, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
cont15 = ['stress_2_gremium run 2','T', condition_names, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

contrast_list = [cont01, cont02, cont03, cont04, cont05, cont06, cont07, cont08, cont09, cont10, cont11, cont12, cont13, cont14, cont15]


In [171]:
condition_names

['stress_1',
 'relax_1',
 'stress_1_gremium',
 'Realign1',
 'Realign2',
 'Realign3',
 'Realign4',
 'Realign5',
 'Realign6',
 'Realign7',
 'Realign8',
 'stress_2',
 'relax_2',
 'stress_2_gremium']

In [172]:
l =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(l)

22

specify nodes

In [173]:
# Nodes used in the nipype beginners guide and the handson example

# SpecifyModel - Generates SPM-specific Model (output: session infos for level1design)
modelspec = Node(SpecifySPMModel(concatenate_runs=False,
                                 input_units='secs',
                                 output_units='secs',
                                 time_repetition=TR,
                                 high_pass_filter_cutoff=256),
                 name="modelspec")

# Level1Design - Generates an SPM design matrix (output: spm mat file)
level1design = Node(Level1Design(bases={'hrf': {'derivs': [0, 0]}}, #basis function (hrf)
                                 timing_units='secs',               # model time derivatives
                                 interscan_interval=TR,
                                 model_serial_correlations='AR(1)'), # model the serial correlation with AR(1) 
                                                                    #(Serial correlations in fMRI time series due to aliased 
                                                                    # biorhythms and unmodelled neuronal activity can be accounted 
                                                                    #for using an autoregressive AR(1) model)
                    name="level1design")

# EstimateModel - estimate the parameters of the model (output: a.o. beta-images)
level1estimate = Node(EstimateModel(estimation_method={'Classical': 1}), # other option would be Bayesian
                      name="level1estimate")

# EstimateContrast - estimates contrasts (output: o.a. con_images (t-maps))
level1conest = Node(EstimateContrast(contrasts=contrast_list), name="level1conest")

specify input and output stream

In [174]:
# Selectfiles - to grab the data
templates = {'anat': '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/brain.mgz', # anatomical images,
             'func': '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/preprocessing/working_dir/preproc_hoaf/_subject_id_{subject_id}/smooth/sr{subject_id}_*.nii',
             'mc_param': '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/preprocessing/working_dir/preproc_hoaf/_subject_id_{subject_id}/realign/rp_{subject_id}*.txt',
             'outliers': '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/preprocessing/working_dir/preproc_hoaf/_subject_id_{subject_id}/art/art.r{subject_id}_*.txt',
             'mask':  '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/preprocessing/working_dir/preproc_hoaf/_subject_id_{subject_id}/wholebrainmask/*.nii',
             #'resp': '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/physio/resp/{subject_id}/*.csv',
             #'puls': '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/physio/puls/{subject_id}/*.csv'
            }


selectfiles = Node(SelectFiles(templates, sort_filelist=True),
          name='selectfiles')

selectfiles.iterables = [('subject_id', subject_list)]

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id', 'contrasts',
                                            'mask'],
                                    contrasts=contrast_list),
                  name="infosource")

infosource.iterables = [('subject_id', subject_list)]

# Initiate the datasink node
output_folder = 'datasink'
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

## Use the following substitutions for the DataSink output
substitutions = [('_subj_id_', 'sub-')]
datasink.inputs.substitutions = substitutions

In [175]:
output_folder

'datasink'

specify contrast

specify GLM

In [176]:
#import pandas as pd
#from os.path import join as opj
#trialinfo = pd.read_table('/data/ds000114/task-fingerfootlips_events.tsv')
#trialinfo

In [178]:
def concat_nui(subject_id):
    
    import pandas as pd
    from os.path import join as opj
    from os.path import basename, abspath
    import numpy as np
    
    path_mc_param = '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/preprocessing/output_dir/realign/%s'%subject_id
    path_resp = '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/physio/resp/%s'%subject_id
    path_pulse = '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/physio/pulse/%s'%subject_id
    
    conf_files = []
    
    for run in ['1', '2']:
        mc_param = pd.read_table(opj(path_mc_param, 'rp_'+subject_id+'_task-stress_'+'run-'+run+'_bold.txt'), header=None, delim_whitespace=True)
        resp_file = pd.read_csv(opj(path_resp, subject_id+'_task-stress_'+'run-'+run+'_physio_resp.csv'), header=None, names=['resp'])
        pulse_file = pd.read_csv(opj(path_pulse, subject_id+'_task-stress_'+'run-'+run+'_physio_pulse.csv'), header=None, names=['pulse'])
   
        conf_df = pd.concat([mc_param, resp_file, pulse_file], axis=1)

        out_file = abspath(basename('confounds_%02d.txt' %int(run)))
    
        np.savetxt(out_file, conf_df.values)
        
        conf_files.append(out_file)
        
        #conf_df = pd.concat([mc_param, resp_file, pulse_file], axis=1)
        #if run == '2':
        #    conf_df_2 = pd.concat([mc_param, resp_file, pulse_file], axis=1)
    
    #list_confs = [conf_df_1, conf_df_2]
    
    return conf_files

In [179]:
# get subject info - get subject specific condition information
getconfinfo = Node(Function(input_names=['subject_id'],
                               output_names=['conf_files'],
                               function=concat_nui),
                      name='getconfinfo')

In [180]:
def get_subject_info(subject_id):   
    
    from nipype.interfaces.base import Bunch
    import pandas as pd
    from os.path import join as opj
    import numpy as np
    
    path = '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/%s/func'%subject_id
    path_resp = '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/physio/resp/%s'%subject_id
    path_pulse = '/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/physio/pulse/%s'%subject_id

    for run in ['1', '2']:
        file = pd.read_table(opj(path, subject_id+'_task-stress_'+'run-'+run+'_events.tsv'))
        resp_file = pd.read_csv(opj(path_resp, subject_id+'_task-stress_'+'run-'+run+'_physio_resp.csv'), header=None, names=['resp'])
        pulse_file = pd.read_csv(opj(path_pulse, subject_id+'_task-stress_'+'run-'+run+'_physio_pulse.csv'), header=None, names=['pulse'])
        # sort stimuli/rows so that they are in the same order like in the behavioral experiments
        if run == '1':
            file['stimuli_cat'] = pd.Categorical(file['trial_type'], categories=['stress_1', 'relax_1',  'stress_1_gremium'], ordered=True)
        elif run == '2':
            file['stimuli_cat'] = pd.Categorical(file['trial_type'], categories=['stress_2', 'relax_2',  'stress_2_gremium'], ordered=True)
        sorted_file = file.sort_values('stimuli_cat')
        sorted_file['stimuli_cat'] = sorted_file['stimuli_cat'].replace(np.nan, 0)
        sorted_file = sorted_file[sorted_file.stimuli_cat  != 0]
        
        #sorted_file.sort_values(by=['stress_gremium'])
        print(sorted_file)
        sorted_file.values.tolist()
        #print(resp_file)
        #print(pulse_file)
        #condition_names = ['stress_1', 'relax_1', 'stress_1_gremium']
        #create lists for each run including the stimuli and their particular onset times
        onset_array = []
        duration_array = []
        stimuli_array = []
        
        for group in sorted_file.groupby('trial_type', sort=False):
            stimuli_array.append(group[0])
            #onset_array.append(sorted(list(round(group[1].onset,1))))
            #duration_array.append(sorted(group[1].duration.tolist()))
            onset_array.append(list(round(group[1].onset,1)))
            duration_array.append(group[1].duration.tolist())
        if run == '1':
            info_run1 = [stimuli_array, onset_array, duration_array]
        elif run == '2':
            info_run2 = [stimuli_array, onset_array, duration_array]
        else:
            print(info_run_error)
            
        #pmod_array = []
        #param_array = []
        
        #if run == '1':
        #    info_pmod_run1 = [Bunch(name=['resp'], param=[[resp_file.resp.tolist()]], 
        #                                  poly=[1])]
        #elif run == '2':
        #    info_pmod_run2 = [Bunch(name=['resp'], param=[[resp_file.resp.tolist()]], 
        #                                  poly=[1])]


    # create subject_info including all runs
    subject_info=[]
    for r in range(2):
        if r == 0:
            subject_info.insert(r,
                                (Bunch(conditions=info_run1[0],
                                 onsets = info_run1[1],
                                 durations = info_run1[2],
                                 amplitudes=None,
                                 tmod=None,
                                 pmod=None)))   
        elif r == 1:
            subject_info.insert(r,
                                (Bunch(conditions=info_run2[0],
                                 onsets = info_run2[1],
                                 durations = info_run2[2],
                                 amplitudes=None,
                                 tmod=None,
                                 pmod=None))) 
        else: 
            print("error2")          
            
    return(subject_info)

In [181]:
get_subject_info('sub-19')

        onset  duration        trial_type condition reaction_sub  \
174  572.7800    2.2190          stress_1  rotation          non   
87   285.9486    3.2645          stress_1      math          non   
187  588.9991    3.1369          stress_1  rotation          non   
211  652.5613    3.3630          stress_1      math          non   
91   292.3994    4.7298          stress_1      math          non   
45   206.0570    1.5211          stress_1  rotation          non   
214  657.3246    2.2224          stress_1      math          non   
94   298.6665    2.5601          stress_1      math          non   
180  579.7985    1.7344          stress_1  rotation          non   
148  538.7575    3.9912          stress_1  rotation          non   
97   302.9165    1.9425          stress_1      math          non   
217  660.9301    2.1925          stress_1      math          non   
35   194.0265    1.7494          stress_1  rotation          non   
145  534.1481    1.1822          stress_1  rotat

[Bunch(amplitudes=None,
       conditions=['stress_1', 'relax_1', 'stress_1_gremium'],
       durations=[[2.218999999999937,
         3.264499999999998,
         3.1368999999999687,
         3.3629999999999427,
         4.729800000000012,
         1.52109999999999,
         2.2223999999999933,
         2.5601000000000345,
         1.7343999999999369,
         3.991200000000049,
         1.9424999999999957,
         2.1924999999999955,
         1.7494000000000085,
         1.1821999999999662,
         4.189200000000029,
         3.250900000000001,
         1.4540000000000075,
         3.4522999999999233,
         1.302500000000009,
         2.403599999999983,
         3.633899999999983,
         3.4566999999999557,
         4.439999999999999,
         3.4060999999999235,
         1.420599999999922,
         1.7630000000000337,
         1.2432000000000016,
         1.8661999999999352,
         1.4641000000000022,
         1.2158000000000015,
         3.899599999999992,
         1.3778999

In [184]:
# get subject info - get subject specific condition information
getsubjectinfo = Node(Function(input_names=['subject_id'],
                               output_names=['subject_info'],
                               function=get_subject_info),
                      name='getsubjectinfo')
#	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-05/_subject_id_sub-05/selectfiles".
#191107-11:42:28,121 nipype.workflow INFO:
#	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")

specify workflow

In [185]:
# Initiation of the 1st-level analysis workflow
l1analysis = Workflow(name='l1analysis')
l1analysis.base_dir = opj(experiment_dir, working_dir)

# Connect up the 1st-level analysis components
l1analysis.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                               ('mask', 'mask')]),
                    (infosource, level1conest, [('contrasts', 'contrasts')]),
                    (infosource, getsubjectinfo, [('subject_id', 'subject_id')]),
                    (infosource, getconfinfo, [('subject_id', 'subject_id')]),
                    (getsubjectinfo, modelspec, [('subject_info',
                                                 'subject_info')]),
                    (selectfiles, modelspec, [('func', 'functional_runs')]),
                    (getconfinfo, modelspec, [('conf_files', 'realignment_parameters')]),
                    #(selectfiles, modelspec, [('atmung', 'event_files')]),
                    #(selectfiles, modelspec, [('puls', 'event_files')]),
                    (selectfiles, level1design, [('mask', 'mask_image')]),
                    (modelspec, level1design, [('session_info',
                                                'session_info')]),
                    (level1design, level1estimate, [('spm_mat_file',
                                                     'spm_mat_file')]),
                    (level1estimate, level1conest, [('spm_mat_file',
                                                     'spm_mat_file'),
                                                    ('beta_images',
                                                     'beta_images'),
                                                    ('residual_image',
                                                     'residual_image')]),
                    (level1estimate, datasink, [('mask_image', 'contrasts.@masks')]),
                    (level1conest, datasink, [('spm_mat_file','contrasts.@spm_mat'),
                                              ('spmT_images', 'contrasts.@T'),
                                              ('con_images', 'contrasts.@con')])])

In [186]:
#concat_nui('sub-05')

visulize the workflow

In [187]:
# Create 1st-level analysis output graph
l1analysis.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
#from IPython.display import Image
#Image('/media/lmn/86A406A0A406933B2/HOAF/HOAF_BIDS/derivatives/firstlevel/output_firstlevel/graph.png')

211014-14:37:01,105 nipype.workflow INFO:
	 Generated workflow graph: /media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/graph.png (graph2use=colored, simple_form=True).


'/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/graph.png'

Run the Workflow

In [188]:
l1analysis.run('MultiProc', plugin_args={'n_procs': 4})

211014-14:37:01,118 nipype.workflow INFO:
	 Workflow l1analysis settings: ['check', 'execution', 'logging', 'monitoring']
211014-14:37:06,100 nipype.workflow INFO:
	 Running in parallel.
211014-14:37:06,181 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 575 jobs ready. Free memory (GB): 28.21/28.21, Free processors: 4/4.
211014-14:37:06,495 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-29/_subject_id_sub-07/selectfiles".
211014-14:37:06,497 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-29/_subject_id_sub-20/selectfiles".

211014-14:37:06,498 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel

/home/lmn/install/miniconda3/lib/python3.6/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:
/home/lmn/install/miniconda3/lib/python3.6/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


211014-14:37:08,514 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-29/_subject_id_sub-09/selectfiles".
211014-14:37:08,515 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-29/_subject_id_sub-23/selectfiles".
211014-14:37:08,519 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-29/_subject_id_sub-25/selectfiles".
211014-14:37:08,520 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
211014-14:37:08,516 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlev

[181 rows x 7 columns]
        onset  duration        trial_type condition reaction_sub  \
237  500.2484    1.7204          stress_2      math          non   
154  381.3634    2.3487          stress_2  rotation          non   
196  460.0871    0.8488          stress_2      math          non   
151  377.6206    2.1135          stress_2  rotation          non   
240  503.2506    0.6511          stress_2      math          non   
184  415.7598    2.4882          stress_2  rotation          non   
212  476.0610    1.2524          stress_2      math          non   
209  472.4078    1.4985          stress_2      math          non   
243  505.9345    0.3669          stress_2      math          non   
145  370.9082    1.4829          stress_2  rotation          non   
199  463.1059    1.0289          stress_2      math          non   
138  361.7883    2.1292          stress_2  rotation          non   
203  467.0939    1.4612          stress_2      math          non   
148  373.9670    1.9514  

	 [Node] Setting-up "l1analysis.level1estimate" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-29/_subject_id_sub-25/level1estimate".
211014-14:37:38,532 nipype.workflow INFO:
	 [Node] Running "level1estimate" ("nipype.interfaces.spm.model.EstimateModel")
211014-14:37:40,228 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 569 jobs ready. Free memory (GB): 27.41/28.21, Free processors: 0/4.
                     Currently running:
                       * l1analysis.level1estimate
                       * l1analysis.level1estimate
                       * l1analysis.level1estimate
                       * l1analysis.level1estimate
211014-14:38:08,142 nipype.workflow INFO:
	 [Node] Finished "l1analysis.level1estimate".
211014-14:38:08,238 nipype.workflow INFO:
	 [Job 35] Completed (l1analysis.level1estimate).
211014-14:38:08,253 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 570 jobs ready. Free 

...

	 [Node] Setting-up "l1analysis.level1conest" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-03/_subject_id_sub-04/level1conest".
211014-17:12:09,619 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.selectfiles" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-03/_subject_id_sub-12/selectfiles".211014-17:12:09,619 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.modelspec" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-03/_subject_id_sub-26/modelspec".

211014-17:12:09,626 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
211014-17:12:09,639 nipype.workflow INFO:
	 [Node] Running "modelspec" ("nipype.algorithms.modelgen.SpecifySPMModel")
211014-17:12:09,641 nipype.workflow INFO:
	 [Node] Finished "l1analysis.selectfil

211014-17:12:31,570 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.level1estimate" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-03/_subject_id_sub-12/level1estimate".

211014-17:12:31,588 nipype.workflow INFO:
	 [Node] Running "level1estimate" ("nipype.interfaces.spm.model.EstimateModel")211014-17:12:31,703 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.level1design" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-03/_subject_id_sub-19/level1design".
211014-17:12:31,952 nipype.workflow INFO:
	 [Node] Running "level1design" ("nipype.interfaces.spm.model.Level1Design")
211014-17:12:33,32 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 4 jobs ready. Free memory (GB): 27.41/28.21, Free processors: 0/4.
                     Currently running:
                       * l1analysis.level1design
                       * l1an

211014-17:13:03,718 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.level1estimate" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-03/_subject_id_sub-28/level1estimate".
211014-17:13:03,732 nipype.workflow INFO:
	 [Node] Running "level1estimate" ("nipype.interfaces.spm.model.EstimateModel")
211014-17:13:05,63 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 3 jobs ready. Free memory (GB): 27.41/28.21, Free processors: 0/4.
                     Currently running:
                       * l1analysis.level1estimate
                       * l1analysis.level1conest
                       * l1analysis.level1conest
                       * l1analysis.level1estimate
211014-17:13:12,713 nipype.workflow INFO:
	 [Node] Finished "l1analysis.level1conest".
211014-17:13:13,56 nipype.workflow INFO:
	 [Job 3182] Completed (l1analysis.level1conest).
211014-17:13:13,73 nipype.workflow INFO:
	 [MultiProc] Running 3 t

211014-17:13:33,647 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.level1conest" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-03/_subject_id_sub-28/level1conest".
211014-17:13:33,822 nipype.workflow INFO:
	 [Node] Running "level1conest" ("nipype.interfaces.spm.model.EstimateContrast")
211014-17:13:34,632 nipype.workflow INFO:
	 [Node] Finished "l1analysis.level1conest".
211014-17:13:35,72 nipype.workflow INFO:
	 [Job 3194] Completed (l1analysis.level1conest).
211014-17:13:35,84 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 2 jobs ready. Free memory (GB): 27.61/28.21, Free processors: 1/4.
                     Currently running:
                       * l1analysis.level1conest
                       * l1analysis.level1design
                       * l1analysis.level1design
211014-17:13:35,288 nipype.workflow INFO:
	 [Node] Finished "l1analysis.level1design".
211014-17:13:35,669 nipype.workflow

211014-17:14:07,385 nipype.workflow INFO:
	 [Node] Finished "l1analysis.level1estimate".
211014-17:14:07,729 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.level1conest" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/working_firstlevel_pmod/l1analysis/_subject_id_sub-03/_subject_id_sub-17/level1conest".
211014-17:14:07,908 nipype.workflow INFO:
	 [Node] Running "level1conest" ("nipype.interfaces.spm.model.EstimateContrast")
211014-17:14:09,96 nipype.workflow INFO:
	 [Job 3211] Completed (l1analysis.level1estimate).
211014-17:14:09,109 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 1 jobs ready. Free memory (GB): 27.81/28.21, Free processors: 2/4.
                     Currently running:
                       * l1analysis.level1conest
                       * l1analysis.level1estimate
211014-17:14:09,704 nipype.workflow INFO:
	 [Node] Setting-up "l1analysis.level1conest" in "/media/lmn/86A406A0A406933B5/HOAF/HOAF_BIDS/derivatives/firstlevel/worki